In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/project/utils.py /content/utils.py

In [ ]:
pip install python-dotenv

In [ ]:
import pandas as pd
import numpy as np
import cv2
from time import perf_counter

import utils

In [ ]:
!cp /content/drive/MyDrive/project/raw_tracks.csv /content/raw_tracks.csv
!cp /content/drive/MyDrive/project/mels.zip /content/mels2.zip
!cp /content/drive/MyDrive/project/files.txt /content/files2.txt

In [ ]:
!cp /content/drive/MyDrive/project/fma_metadata/tracks.csv /content/tracks.csv
!cp /content/drive/MyDrive/project/fma_metadata/genres.csv /content/genres.csv
!cp /content/drive/MyDrive/project/fma_metadata/features.csv /content/features.csv
!cp /content/drive/MyDrive/project/fma_metadata/echonest.csv /content/echonest.csv

In [ ]:
!unzip mels2.zip -d .

In [ ]:
DOWNSAMPLE_HEIGHT = 128
DOWNSAMPLE_WIDTH = 1291

In [ ]:
import ast
f = open("files2.txt", "r")
data = f.read()
indices = ast.literal_eval(data)
f.close()

In [ ]:
track_data = pd.read_csv("raw_tracks.csv", index_col = 0)
data = track_data["track_genres"][np.array(indices).astype(int)]
# track_data["track_genres"]

In [ ]:
data_dict = dict()
for i in data.keys():
    data_dict[i] = ast.literal_eval(data[i])[0]["genre_title"] #data_dict contains track_ids as keys and genre_ids as values

In [ ]:
tracks = utils.load('drive/MyDrive/project/fma_metadata/tracks.csv')

In [ ]:
small = tracks['set', 'subset'] <= 'small'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train_initial = tracks.loc[small & train, ('track', 'genre_top')]
y_test_initial = tracks.loc[small & test, ('track', 'genre_top')]
y_val_initial = tracks.loc[small & val, ('track', 'genre_top')]

y_train_initial = y_train_initial.to_frame()
y_test_initial = y_test_initial.to_frame()
y_val_initial = y_val_initial.to_frame()

genres = ('Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock')
genres_dic = dict()
cnt = 0
for i in genres:
  genres_dic[i] = cnt
  cnt += 1

In [ ]:
y_train_np = y_train_initial.to_numpy()
y_test_np = y_test_initial.to_numpy()
y_val_np = y_val_initial.to_numpy()
y_train_np[0,0]

'Hip-Hop'

In [ ]:
x_train = np.zeros((63940, 128, 128))
x_test = np.zeros((8000 , 128, 128))
x_val = np.zeros((8000 , 128, 128))

y_train = np.zeros(63940, dtype = int)
y_test = np.zeros(8000, dtype = int)
y_val = np.zeros(8000, dtype = int)

i1, i2, i3, count = 0, 0, 0, 0
for i in data_dict:
    file = "{:06d}".format(i)
    dir = file[:3]
    track_id = int(file)
    img = cv2.imread(f"/content/mels/{dir}/{file}.png")
    if img.shape[1] > 1291:
      img = np.delete(img, np.s_[1291:img.shape[1]], axis=1)
    img = cv2.resize(img, (DOWNSAMPLE_WIDTH, DOWNSAMPLE_HEIGHT), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) 
    if track_id in y_train_initial.index:
      for j in range(10):
        x_train[10*i1+j] = img[:,128*j:128*(j+1)]
        y_train[10*i1+j] = genres_dic[y_train_np[i1,0]]
      i1 += 1
      count += 1
    elif track_id in y_test_initial.index:
      for j in range(10):
        x_test[10*i2+j] = img[:,128*j:128*(j+1)]
        y_test[10*i2+j] = genres_dic[y_test_np[i2,0]]
      i2 += 1
      count += 1
    elif track_id in y_val_initial.index:
      for j in range(10):
        x_val[10*i3+j] = img[:,128*j:128*(j+1)]
        y_val[10*i3+j] = genres_dic[y_val_np[i3,0]]
      i3 += 1
      count += 1
print(i1,i2,i3,count)      

IndexError: ignored

In [ ]:
y_train.shape, y_test.shape, y_val.shape

In [ ]:
i1, i2, i3

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(x_train[7378], cmap='gray')
# print(img_datas[3].shape)

In [ ]:
# img_datas = normalize(img_datas)
# print(img_datas.shape)

# tracks = utils.load('tracks.csv')
# genres = utils.load('genres.csv')
# features = utils.load('features.csv')
# echonest = utils.load('echonest.csv')

# tracks.shape, genres.shape, features.shape, echonest.shape

KNN

In [ ]:
import math
from time import perf_counter
def test(train_x, train_y, test_x, test_y, k):
  start_time = perf_counter()
  correct = [0] * 11
  for i in range(test_x.shape[0]):  
    distances = np.zeros(train_x.shape[0])
    tot = 0
    for j in range(train_x.shape[0]):
      distances[j] = math.sqrt(np.sum(np.power((test_x[i]-train_x[j]), 2)))
    distances = np.argsort(distances)
    predict = [0] * 11
    for q in range(11):
        predict[q] = np.bincount(train_y[distances[0:2*q+3]]).argmax()
        correct[q] += int(predict[q] == test_y[i])   
    if not i%8:
        print("\r{:3d}% (Time elapsed: {:3d}s)".format(i//8, int((perf_counter()-start_time)//1)), end = "")
  acc = [0] * 11
  print()
  for q in range(11):
      acc[q]  = (correct[q]/test_x.shape[0])*100 
  return acc

In [ ]:
def test2(train_x, train_y, test_x, test_y, k):
  start_time = perf_counter()
  correct = [0] * 11
  for i in range(test_x.shape[0]):  
    distances = np.zeros(train_x.shape[0])
    tot = 0
    for j in range(train_x.shape[0]):
      distances[j] = np.sum(np.abs((test_x[i]-train_x[j])))
    distances = np.argsort(distances)
    predict = [0] * 11
    for q in range(11):
        predict[q] = np.bincount(train_y[distances[0:2*q+3]]).argmax()
        correct[q] += int(predict[q] == test_y[i])    
    if not i%8:
        print("\r{:3d}% (Time elapsed: {:3d}s)".format(i//8, int((perf_counter()-start_time)//1)), end = "")
  acc = [0] * 11
  print()
  for q in range(11):
      acc[q]  = (correct[q]/test_x.shape[0])*100 
  return acc

In [ ]:
x_train /= 255
x_test /= 255
x_val /= 255

In [ ]:
euclidean_arr = test(x_train.reshape(6394, -1), y_train, x_test.reshape(800, -1), y_test, 9)
print("\n", euclidean_arr)
manhattan_arr = test2(x_train.reshape(6394, -1), y_train, x_test.reshape(800, -1), y_test, 9)
print("\n", manhattan_arr) 

 99% (Time elapsed: 2805s)

 [21.125, 21.0, 21.625, 23.25, 23.875, 23.5, 23.625, 23.625, 23.125, 23.75, 23.625]
 99% (Time elapsed: 1016s)

 [20.625, 22.625, 22.375, 23.0, 23.375, 24.0, 24.125, 23.625, 24.0, 24.375, 23.25]


In [ ]:
print("Results for image resolution 32x322:")
print("Euclidean Accuracy: 21.375, 20.875, 24.250, 23.375, 23.000, 23.750, 24.000, 24.500, 23.625, 23.375, 23.750")
print("k_val:                   3,      5,      7,      9,     11,     13,     15,     17,     19,     21,     23")
print("Manhattan Accuracy: 21.500, 22.375, 24.125, 24.250, 24.500, 24.125, 23.375, 23.375, 22.875, 23.250, 24.250")

Euclidean Accuracy: 21.375, 20.875, 24.250, 23.375, 23.000, 23.750, 24.000, 24.500, 23.625, 23.375, 23.750
k_val:                   3,      5,      7,      9,     11,     13,     15,     17,     19,     21,     23
Manhattan Accuracy: 21.500, 22.375, 24.125, 24.250, 24.500, 24.125, 23.375, 23.375, 22.875, 23.250, 24.250


In [ ]:
x_train_cn = np.expand_dims(x_train, axis=3)
x_test_cn = np.expand_dims(x_test, axis=3)
x_val_cn = np.expand_dims(x_val, axis=3)

In [ ]:
y_train_cn = np.zeros((y_train.size, y_train.max()+1))
y_train_cn[np.arange(y_train.size),y_train] = 1

y_test_cn = np.zeros((y_test.size, y_test.max()+1))
y_test_cn[np.arange(y_test.size),y_test] = 1

y_val_cn = np.zeros((y_val.size, y_val.max()+1))
y_val_cn[np.arange(y_val.size),y_val] = 1

In [ ]:
import tensorflow as tf
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform

from keras.preprocessing.image import ImageDataGenerator
import random

In [ ]:
def GenreModel(input_shape = (128,128,1),classes=8):
  
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(8,8),strides=(1,1))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  #X = Dropout(rate=0.3)(X)

  
  X = Conv2D(16,kernel_size=(4,4),strides = (1,1))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  #X = Dropout(rate=0.3)(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(4,4),strides = (1,1))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  #X = Dropout(rate=0.3)(X)

  X = Conv2D(64,kernel_size=(4,4),strides=(1,1))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  #X = Dropout(rate=0.3)(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(128,kernel_size=(4,4),strides=(1,1))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  #X = Dropout(rate=0.3)(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)
  X = Dropout(rate=0.5)(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model

In [ ]:
model = GenreModel(input_shape=(128,128,1),classes=8)
model.summary()

Model: "GenreModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 121, 121, 8)       520       
                                                                 
 batch_normalization_5 (Batc  (None, 121, 121, 8)      32        
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 121, 121, 8)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 118, 118, 16)      2064      
                                                                 
 batch_normalization_6 (Batc  (None, 118, 118, 16)     64        
 hNormalization)                                        

In [ ]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

model.fit(x_train_cn, y_train_cn,epochs=70, batch_size = 128, validation_data=(x_test_cn, y_test_cn))    

Epoch 1/70
500/500 [==============================] - 44s 65ms/step - loss: 2.2490 - accuracy: 0.2288 - get_f1: 0.0361 - val_loss: 2.0945 - val_accuracy: 0.1937 - val_get_f1: 0.0039
Epoch 2/70
500/500 [==============================] - 31s 62ms/step - loss: 1.8778 - accuracy: 0.3123 - get_f1: 0.0663 - val_loss: 2.0756 - val_accuracy: 0.2351 - val_get_f1: 0.0792
Epoch 3/70
500/500 [==============================] - 31s 62ms/step - loss: 1.8262 - accuracy: 0.3449 - get_f1: 0.1047 - val_loss: 2.3776 - val_accuracy: 0.1482 - val_get_f1: 0.1144
Epoch 4/70
500/500 [==============================] - 31s 62ms/step - loss: 1.7869 - accuracy: 0.3635 - get_f1: 0.1372 - val_loss: 2.0428 - val_accuracy: 0.2769 - val_get_f1: 0.1494
Epoch 5/70
500/500 [==============================] - 31s 62ms/step - loss: 1.7468 - accuracy: 0.3802 - get_f1: 0.1689 - val_loss: 1.9516 - val_accuracy: 0.2719 - val_get_f1: 0.0517
Epoch 6/70
500/500 [==============================] - 31s 62ms/step - loss: 1.7081 - accur